In [ ]:
from zizou.data import WaveformBaseclass
from datetime import datetime, date, timedelta
import pandas as pd
import requests
from io import StringIO

In [ ]:
def tilde_request(domain: str, name: str, station: str,
                  method: str, sensor: str, aspect: str,
                  startdate: date, enddate: date) -> pd.DataFrame:
    """
    Request data from the tilde API (https://tilde.geonet.org.nz/v3/api-docs/).
    See the tilde discovery tool for more information:
    https://tilde.geonet.org.nz/ui/data-discovery/

    Parameters
    ----------
    domain : str
        The domain of the data (e.g. 'manualcollect')
    name : str
        The name of the data (e.g. 'plume-SO2-gasflux')
    station : str
        The station code (e.g. 'WI000')
    method : str
        The method of the data (e.g. 'contouring')
    sensor : str
        The sensor of the data (e.g. 'MC01')
    aspect : str
        The aspect of the data (e.g. 'nil')
    startdate : date
        The start date of the data
    enddate : date
        The end date of the data

    Returns
    -------
    pd.DataFrame
        A pandas dataframe with the requested data
    """
    tilde_url = "https://tilde.geonet.org.nz/v3/data"
    _tstart = str(startdate)
    # split the request into historic and latest data
    get_historic = True
    _tend = str((enddate - timedelta(days=29)))
    if startdate > (enddate - timedelta(days=29)):
        get_historic = False

    latest = f"{tilde_url}/{domain}/{station}/{name}/{sensor}/{method}/{aspect}/latest/30d"
    rval = requests.get(latest, headers={'Accept':'text/csv'})
    if rval.status_code != 200:
        msg = f"Download of {name} for {station} failed with status code {rval.status_code}"
        msg += f" and url {latest}"
        raise ValueError(msg)
    df_latest = pd.read_csv(StringIO(rval.text), index_col='timestamp', parse_dates=['timestamp'],
                            usecols=['timestamp', 'value', 'error'],
                            date_format='ISO8601')
    if get_historic:
        historic = f"{tilde_url}/{domain}/{station}/{name}/{sensor}/{method}/{aspect}/"
        historic += f"{_tstart}/{_tend}"
        rval = requests.get(historic, headers={'Accept':'text/csv'})
        if rval.status_code != 200:
            msg = f"Download of {name} for {station} failed with status code {rval.status_code}"
            raise ValueError(msg)
        data = StringIO(rval.text)
        df_historic = pd.read_csv(data, index_col='timestamp', parse_dates=['timestamp'],
                                usecols=['timestamp', 'value', 'error'],
                                date_format='ISO8601')
        if len(df_latest) > 0:
            df = df_historic.combine_first(df_latest)
        else:
            df = df_historic
    df.rename(columns={'value': 'obs', 'error': 'err'}, inplace=True)
    df.index.name = 'dt'
    return df

In [ ]:
tilde_request(domain='geomag', name='magnetic-field', station='EYWM', method='-',
              sensor='-', aspect='-', startdate=date(2024, 7, 1), enddate=date(2024, 8, 21))

In [ ]:
class GeoMagWaveforms(WaveformBaseclass):

    def __init__(self, base_url, method: str, aspect: str,
                 name: str):
        self.base_url = base_url
        self.method = method
        self.aspect = aspect
        self.name = name
        self.domain = 'geomag'

    def get_waveform(self, net: str, site: str, loc: str, comp: str,
                     startdate: date, enddate: date) -> pd.DataFrame:
        df = tilde_request(domain=self.domain, name=self.name, station=site,
                           method=self.method, sensor=loc, aspect=self.aspect,
                           startdate=startdate, enddate=enddate)
        return df

gmc = GeoMagWaveforms(base_url='https://tilde.geonet.org.nz/v3/data',
                      method='-', aspect='-', name='magnetic-field')
df = gmc.get_waveform(net='', site='EYWM', loc='-', comp='',
                      startdate=date(2024, 7, 1), enddate=date(2024, 8, 21))

In [ ]:
from obspy import Trace, UTCDateTime
tr = Trace?

In [ ]:
df.shape